<a href="https://colab.research.google.com/github/u6yuvi/NER-System/blob/main/notebooks/Cross_Encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ntropy-network/ML-tools.git

Cloning into 'ML-tools'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 131 (delta 46), reused 110 (delta 35), pack-reused 0
Receiving objects: 100% (131/131), 779.41 KiB | 17.32 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [2]:
!pip3 install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.6/518.6 KB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.3 MB/s eta 0:00:00


In [3]:
!pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.5 MB/s eta 0:00:00


In [4]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [5]:
if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!ls

drive  ML-tools  sample_data


In [7]:
%cd ML-tools/parallel_cross_encoder/

/content/ML-tools/parallel_cross_encoder


In [8]:
!ls

parallel_cross_encoder	pyproject.toml	Test.ipynb  Train.ipynb
poetry.lock		README.md	tests


In [37]:
from parallel_cross_encoder.configs import TrainConfigs
from copy import deepcopy
from torch.utils.data.dataloader import DataLoader

# Training Configs

In [10]:
from typing import List

from psutil import cpu_count
from pydantic import BaseModel, ValidationError, validator


class TrainConfigs(BaseModel):
    no_cuda: bool = False
    parallelize_cross_encoder: bool = True
    validation_data_ratio: float = 0.3
    output_dir: str = "outputs"
    batch_size: int = 32
    weight_decay: float = 0.0001
    dataloader_num_workers: int = 2
    model_type: str = "bert"
    negative_labels_per_sample: int = 3  # For parralelized cross encoder only
    early_stopping_epochs: int = 6
    warmup_epochs: float = 1
    encoder_hidden_layers: int = 2
    decoder_hidden_layers: int = 1  # For bart model only
    epochs: int = 10
    validation_threshold: float = 0.5
    learning_rate: float = 5e-5
    weight_decay: float = 1e-2
    logging_steps: int = 50
    max_length: int = 256

    @validator("model_type")
    def validate_model_type(cls, v):
        if v not in ["bert", "bart"]:
            raise ValidationError('Value must be one of ["bert", "bart"]')
        return v

    @validator("dataloader_num_workers")
    def validate_dataloader_num_workers(cls, v):
        if v is None or v == -1:
            return cpu_count()
        return v


class TestConfigs(BaseModel):
    device: str = "cpu"
    model_path: str
    batch_size: int = 16
    num_labels_per_sample: int = 4  # For parallelized cross-encoders only
    dataloader_num_workers: int = 2

    @validator("dataloader_num_workers")
    def validate_dataloader_num_workers(cls, v):
        if v is None or v == -1:
            return cpu_count()
        return v

In [11]:
configs = TrainConfigs(no_cuda=True, parallelize_cross_encoder=False, model_type='bert', output_dir="outputs", encoder_hidden_layers=2, epochs=6, batch_size=32, dataloader_num_workers=2, learning_rate=1e-04, early_stopping_epochs=6, validation_data_ratio=0.15, max_length=128)
configs

TrainConfigs(no_cuda=True, parallelize_cross_encoder=False, validation_data_ratio=0.15, output_dir='outputs', batch_size=32, weight_decay=0.01, dataloader_num_workers=2, model_type='bert', negative_labels_per_sample=3, early_stopping_epochs=6, warmup_epochs=1, encoder_hidden_layers=2, decoder_hidden_layers=1, epochs=6, validation_threshold=0.5, learning_rate=0.0001, logging_steps=50, max_length=128)

In [12]:
if configs.parallelize_cross_encoder:
    name = "parallelized_cross_encoder"
else:
  name = "vanilla_cross_encoder"
name = name + "_" + configs.model_type

name

'vanilla_cross_encoder_bert'

In [13]:
configs

TrainConfigs(no_cuda=True, parallelize_cross_encoder=False, validation_data_ratio=0.15, output_dir='outputs', batch_size=32, weight_decay=0.01, dataloader_num_workers=2, model_type='bert', negative_labels_per_sample=3, early_stopping_epochs=6, warmup_epochs=1, encoder_hidden_layers=2, decoder_hidden_layers=1, epochs=6, validation_threshold=0.5, learning_rate=0.0001, logging_steps=50, max_length=128)

In [14]:
from transformers.models.distilbert.modeling_distilbert import DistilBertForSequenceClassification
from transformers import AutoModel, AutoTokenizer

In [15]:
#DistilBert pooling Layer
import torch
#bs,seq_len,dim
hidden_state = torch.tensor([[[2]*768]*512]*32)
print(hidden_state.shape)
hidden_state[:0] #taking the output representation from CLS token


torch.Size([32, 512, 768])


tensor([], size=(0, 512, 768), dtype=torch.int64)

In [16]:
def load_model(configs):
  model_type = configs.model_type
  encoder_hidden_layers = configs.encoder_hidden_layers
  decoder_hidden_layers = configs.decoder_hidden_layers
  multilabel = configs.parallelize_cross_encoder
  print(multilabel,model_type)
  if model_type == "bert":
      model_name = "typeform/distilbert-base-uncased-mnli"
      if multilabel:
          #model_class = DistilBertForMultiSequenceClassification
          pass
      else:
          model_class = DistilBertForSequenceClassification
      aux = AutoModel.from_pretrained(model_name)
      config = deepcopy(aux.config)
      if encoder_hidden_layers == -1:
         encoder_hidden_layers = config.num_hidden_layers
      config._num_labels = 2
      config.label2id = {"contradiction": 0, "entailment": 1}
      config.id2label = {0: "contradiction", 1: "entailment"}
      config.classif_dropout = 0.2
      config.classifier_dropout = 0.2
      if encoder_hidden_layers<1:
        raise ValueError(f'Encoder Hidden Layers {encoder_hidden_layers} not supported. It should be greater than 0')
      config.num_hidden_layers = encoder_hidden_layers #config part of PreTrained Config https://github.com/huggingface/transformers/blob/main/src/transformers/configuration_utils.py#L48:7
      model = model_class(config)

  tokenizer = AutoTokenizer.from_pretrained(model_name)

  model.cls_token_id = tokenizer.cls_token_id
  model.pad_token_id = tokenizer.pad_token_id
  model.sep_token_id = tokenizer.sep_token_id

  aux_state_dict = aux.state_dict()

  from collections import OrderedDict

  tag = "model."
  new_dict = OrderedDict()
  for k, v in model.state_dict().items():
      if k[len(tag) :] in aux_state_dict:
          if "classif" not in k:
              new_dict[k] = deepcopy(aux_state_dict[k[len(tag) :]])
          else:
              new_dict[k] = v
      else:
          new_dict[k] = v
  model.load_state_dict(new_dict)
  del aux
  return model, tokenizer

In [17]:
model, tokenizer = load_model(configs)

False bert


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Some weights of the model checkpoint at typeform/distilbert-base-uncased-mnli were not used when initializing DistilBertModel: ['pre_classifier.bias', 'classifier.bias', 'classifier.weight', 'pre_classifier.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [18]:
# Vanilla Distilbert

# model_type = configs.model_type
# encoder_hidden_layers = configs.encoder_hidden_layers
# decoder_hidden_layers = configs.decoder_hidden_layers
# multilabel = configs.parallelize_cross_encoder
# print(multilabel,model_type)
# if model_type == "bert":
#     model_name = "typeform/distilbert-base-uncased-mnli"
#     if multilabel:
#         #model_class = DistilBertForMultiSequenceClassification
#         pass
#     else:
#         model_class = DistilBertForSequenceClassification
#     aux = AutoModel.from_pretrained(model_name)
#     config = deepcopy(aux.config)
#     config._num_labels = 2
#     config.label2id = {"contradiction": 0, "entailment": 1}
#     config.id2label = {0: "contradiction", 1: "entailment"}
#     config.classif_dropout = 0.2
#     config.classifier_dropout = 0.2
#     if encoder_hidden_layers<1:
#       raise ValueError(f'Encoder Hidden Layers {encoder_hidden_layers} not supported. It should be greater than 0')
#     config.num_hidden_layers = encoder_hidden_layers #config part of PreTrained Config https://github.com/huggingface/transformers/blob/main/src/transformers/configuration_utils.py#L48:7
#     model = model_class(config)

# tokenizer = AutoTokenizer.from_pretrained(model_name)

# model.cls_token_id = tokenizer.cls_token_id
# model.pad_token_id = tokenizer.pad_token_id
# model.sep_token_id = tokenizer.sep_token_id

# aux_state_dict = aux.state_dict()

# from collections import OrderedDict

# tag = "model."
# new_dict = OrderedDict()
# for k, v in model.state_dict().items():
#     if k[len(tag) :] in aux_state_dict:
#         if "classif" not in k:
#             new_dict[k] = deepcopy(aux_state_dict[k[len(tag) :]])
#         else:
#             new_dict[k] = v
#     else:
#         new_dict[k] = v
# model.load_state_dict(new_dict)



In [19]:
#model.state_dict().keys(), #aux.state_dict().keys()

In [20]:
# for k, v in aux_state_dict.items():
#   print(k)

# Dataset

In [21]:
!pip3 install datasets
from datasets import load_dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 18.8 MB/s eta 0:00:00


In [22]:
newsgroups_data = load_dataset("ag_news")["train"]
newsgroups_data = newsgroups_data.shuffle(seed=42)


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
split = int(len(newsgroups_data) * (1 - configs.validation_data_ratio))
train_data, val_data = newsgroups_data[:split], newsgroups_data[split:]

In [24]:
train_data.keys()

dict_keys(['text', 'label'])

In [25]:
train_data["text"][0]

'Bangladesh paralysed by strikes Opposition activists have brought many towns and cities in Bangladesh to a halt, the day after 18 people died in explosions at a political rally.'

In [26]:
train_data["label"][0]

0

In [27]:
LABELS = [
    "world",
    "sports",
    "business",
    "science and technology",
]

HYPOTHESIS = "This example is about"

In [28]:
2//len(train_data["label"])

0

In [29]:
len(train_data["label"])

102000

In [30]:
class NewsDatasetSingleLabel:
    """
    This is the basic dataset for the vanilla cross-encoder (1 label per sample)
    """

    def __init__(
        self,
        samples,
        targets,
        target_names,
        tokenizer,
        model_type,
        max_length,
        entailment_id=1,
        contradiction_id=0,
    ):
        self.samples = samples
        self.targets = targets
        self.target_names = target_names

        self.model_type = model_type
        self.max_length = max_length
        self.entailment_id = entailment_id
        self.contradiction_id = contradiction_id
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.samples) * len(self.target_names)

    def __getitem__(self, idx):
        sample_idx = idx // len(self.target_names)
        label_idx = idx % len(self.target_names)
        prompt = self.samples[sample_idx]
        target_idx = self.targets[sample_idx]

        label = self.target_names[label_idx]
        if target_idx == label_idx:
            # pos label
            target = [self.entailment_id]
        else:
            # neg label
            target = [self.contradiction_id]

        hypothesis = f"{HYPOTHESIS} {label}"
        enc = self.tokenizer(
            prompt,
            hypothesis,
            truncation="only_first",
            max_length=self.max_length,
            return_tensors="pt",
        )
        enc["labels"] = torch.tensor(target, dtype=torch.long)
        enc["sample_idxs"] = torch.tensor(sample_idx, dtype=torch.long)

        return {k: v.squeeze() for k, v in enc.items()}

In [31]:
def load_train_data(tokenizer,configs):
  ds_train = NewsDatasetSingleLabel(
      samples=train_data["text"],
      targets=train_data["label"],
      target_names=LABELS,
      tokenizer=tokenizer,
      model_type=configs.model_type,
      max_length=configs.max_length,
  )
  ds_val = NewsDatasetSingleLabel(
      samples=val_data["text"],
      targets=val_data["label"],
      target_names=LABELS,
      tokenizer=tokenizer,
      model_type=configs.model_type,
      max_length=configs.max_length,
  )
  return ds_train, ds_val

In [35]:
from copy import deepcopy
from typing import OrderedDict

import numpy as np
import pytorch_lightning as pl
import torch

from parallel_cross_encoder.configs import TrainConfigs
from parallel_cross_encoder.dataset import NewsDatasetInferenceMultiLabel
from parallel_cross_encoder.models.bart_model import BartForMultiSequenceClassification
from parallel_cross_encoder.models.bert_model import DistilBertForMultiSequenceClassification

from psutil import cpu_count
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
from transformers import (
    AdamW,
    AutoModel,
    AutoTokenizer,
    BartForSequenceClassification,
    DistilBertForSequenceClassification,
    get_linear_schedule_with_warmup,
)
from transformers.data.data_collator import DataCollatorWithPadding


class LightningModel(pl.LightningModule):
    """
    Lightning Model for the vanilla/parallelized cross-encoders
    for faster transformers training and inference.
    """

    def __init__(self, configs: TrainConfigs):
        super().__init__()
        if isinstance(configs, dict):
            configs = TrainConfigs(**configs)
        self.configs = configs
        print("MODEL TRAINED CONFIGS :", configs)
        self.save_hyperparameters(configs.dict())
        self.model, self.tokenizer = load_model(configs)
        self.steps_per_epoch = None  # Need to be setup after init, manually

    def forward(self, input_ids, attention_mask, labels=None):
        if len(input_ids.shape) == 3:
            input_ids = input_ids.squeeze(0)
            attention_mask = attention_mask.squeeze(0)
            if labels is not None:
                labels = labels.squeeze(0)
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
        )
        # take only the entailment output in case of non-parallel model:
        if not self.configs.parallelize_cross_encoder:
            outputs.logits = outputs.logits[:, 1]
        return outputs

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        outputs = self(input_ids, attention_mask, labels)
        self.log("train_loss", outputs.loss.item(), prog_bar=True, logger=True)
        return outputs.loss

    def validation_step(self, batch, batch_idx, *args, **kwargs):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        outputs = self(input_ids, attention_mask, labels)
        self.log("val_loss", outputs.loss, prog_bar=True, logger=True)
        return {
            "loss": outputs.loss,
            "logits": outputs.logits.view(-1),
            "labels": labels.view(-1),
        }

    def validation_epoch_end(self, out):
        logits = torch.cat([x["logits"] for x in out])
        labels = torch.cat([x["labels"] for x in out])
        probs = torch.sigmoid(logits)
        preds = probs > self.configs.validation_threshold
        labels = labels.detach().cpu()
        preds = preds.detach().cpu()
        precision, recall, f1, _ = precision_recall_fscore_support(
            labels, preds, average="binary", zero_division=0
        )
        acc = round(accuracy_score(labels, preds), 4)
        precision = round(precision, 4)
        recall = round(recall, 4)
        f1 = round(f1, 4)
        self.log("val_accuracy", acc)
        self.log("val_precision", precision)
        self.log("val_recall", recall)
        self.log("val_f1", f1)
        print(f"Validation :: acc: {acc}, pre: {precision}, rec: {recall}, f1: {f1}")
        return out

    def configure_optimizers(self):
        """Prepare optimizer and scheduler (linear warmup and weight decay)"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [
                    p
                    for n, p in model.named_parameters()
                    if not any(nd in n for nd in no_decay)
                ],
                "weight_decay": self.configs.weight_decay,
            },
            {
                "params": [
                    p
                    for n, p in model.named_parameters()
                    if any(nd in n for nd in no_decay)
                ],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(
            optimizer_grouped_parameters,
            lr=self.configs.learning_rate,
        )

        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=self.steps_per_epoch * self.configs.warmup_epochs,
            num_training_steps=self.steps_per_epoch * self.configs.epochs,
        )
        scheduler = {"scheduler": scheduler, "interval": "step", "frequency": 1}
        return [optimizer], [scheduler]

    def predict(
        self, dataset, batch_size=32, dataloader_num_workers=-1, device="cpu"
    ) -> list:
        """Predict labels on a given dataset.
        Works for vanilla or parallelized cross-encoders.
        Args:
            dataset (torch.Dataset): any torch NLP dataset
            batch_size (int, optional): Batch size. Defaults to 32.
            dataloader_num_workers (int, optional): Num workers (-1 for MAX). Defaults to -1.
            device (str, optional): (device cuda or cpu). Defaults to "cpu".
        Returns:
            list: predictions
        """

        self.eval()
        if dataloader_num_workers == -1:
            dataloader_num_workers = cpu_count()
        scores = [[] for _ in range(len(dataset.samples))]

        if isinstance(dataset, NewsDatasetInferenceMultiLabel):
            loader = torch.utils.data.DataLoader(
                dataset,
                num_workers=dataloader_num_workers,
                batch_size=1,
                drop_last=False,
            )
        else:
            loader = torch.utils.data.DataLoader(
                dataset,
                num_workers=dataloader_num_workers,
                collate_fn=DataCollatorWithPadding(
                    self.tokenizer, pad_to_multiple_of=2
                ),
                batch_size=batch_size,
                drop_last=False,
            )

        for batch in tqdm(loader):
            with torch.no_grad():
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                sample_idxs = batch["sample_idxs"].detach().cpu().numpy()
                out = self(input_ids, attention_mask)
                out = out.logits.detach().cpu().numpy()
                yhat = 1 / (1 + np.exp(-out))
                if len(yhat.shape) == 1:
                    yhat = np.expand_dims(yhat, axis=1)
                if isinstance(dataset, NewsDatasetInferenceMultiLabel):
                    labels_mask = batch["labels_mask"].detach().cpu().numpy()[0]
                    for i, idx in enumerate(sample_idxs[0]):
                        yhat_no_padding = yhat[i][labels_mask[i] != 0]
                        scores[idx].extend(list(yhat_no_padding))
                else:
                    [scores[idx].append(yhat[i]) for i, idx in enumerate(sample_idxs)]

        preds = [np.argmax(s) for s in scores]

        return preds


In [36]:
if configs.no_cuda:
  trainer = pl.Trainer(
      #logger=logger,
      #enable_checkpointing=checkpoint_callback,
      #callbacks=callbacks,
      max_epochs=configs.epochs,
      accelerator="cpu",
      log_every_n_steps=configs.logging_steps,
          )

else:
  trainer = pl.Trainer(
      #logger=logger,
      #enable_checkpointing=checkpoint_callback,
      #callbacks=callbacks,
      max_epochs=configs.epochs,
      gpus=-1,
      accelerator="auto",
      log_every_n_steps=configs.logging_steps,)
model = LightningModel(configs)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


MODEL TRAINED CONFIGS : no_cuda=True parallelize_cross_encoder=False validation_data_ratio=0.15 output_dir='outputs' batch_size=32 weight_decay=0.01 dataloader_num_workers=2 model_type='bert' negative_labels_per_sample=3 early_stopping_epochs=6 warmup_epochs=1 encoder_hidden_layers=2 decoder_hidden_layers=1 epochs=6 validation_threshold=0.5 learning_rate=0.0001 logging_steps=50 max_length=128
False bert


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Some weights of the model checkpoint at typeform/distilbert-base-uncased-mnli were not used when initializing DistilBertModel: ['pre_classifier.bias', 'classifier.bias', 'classifier.weight', 'pre_classifier.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument h

In [ ]:
ds_train, ds_val = load_train_data(tokenizer, configs)
train_dataloader = DataLoader(
    ds_train,
    batch_size=configs.batch_size,
    collate_fn=DataCollatorWithPadding(tokenizer, pad_to_multiple_of=2),
    shuffle=True,
    drop_last=True,
    num_workers=configs.dataloader_num_workers,
)
val_dataloader = DataLoader(
    ds_val,
    batch_size=configs.batch_size,
    collate_fn=DataCollatorWithPadding(tokenizer, pad_to_multiple_of=2),
    shuffle=False,
    drop_last=False,
    num_workers=configs.dataloader_num_workers,
)
# setup line:
model.steps_per_epoch = len(train_dataloader)
print(f"Training for {configs.epochs} epochs ...")
trainer.fit(
    model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader
)


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                                | Params
--------------------------------------------------------------
0 | model | DistilBertForSequenceClassification | 38.6 M
--------------------------------------------------------------
38.6 M    Trainable params
0         Non-trainable params
38.6 M    Total params
154.414   Total estimated model params size (MB)


Training for 6 epochs ...


Sanity Checking: 0it [00:00, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Validation :: acc: 0.75, pre: 0.0, rec: 0.0, f1: 0.0


Training: 0it [00:00, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
